# Gradio Day!

Today we will build User Interfaces using the outrageously simple Gradio framework.

Prepare for joy!

In [1]:
import os

# The requests library is used to make HTTP requests to retrieve the contents of a web page. 
# In this project, it will be used to fetch the HTML of the page that needs to be summarized.
# This library simplifies making HTTP requests, such as GET and POST, handling network errors, 
# and providing access to response content in different formats (text, JSON, etc.).
# This step is essential to retrieve and prepare text data before summarization using GPT-4.
import requests

# BeautifulSoup is part of the bs4 library and is used to parse and extract data from HTML and XML documents. 
# After fetching the web page content using requests, BeautifulSoup helps extract clean, readable text from the raw HTML, 
# which can then be summarized by the GPT-4 model.
from bs4 import BeautifulSoup

# Python's typing module allows you to specify types for variables & function arguments, improving code readability and making it easier to catch errors.
# Certain functions in this project expect lists as input, providing more clarity on data structures passed between functions.
from typing import List

from dotenv import load_dotenv

from openai import OpenAI
import google.generativeai
import anthropic

In [6]:
# Run the following code in your Jupyter environment to check the installed version of Pydantic
# If you have Pydantic v2.x installed, try downgrading it to a 1.x version, which is likely compatible with Gradio.
!pip show pydantic

Name: pydantic
Version: 1.10.7
Summary: Data validation and settings management using python type hints
Home-page: https://github.com/pydantic/pydantic
Author: Samuel Colvin
Author-email: s@muelcolvin.com
License: MIT
Location: D:\Programms\Lib\site-packages
Requires: typing-extensions
Required-by: anaconda-cloud-auth, anthropic, fastapi, google-generativeai, gradio, openai


To install Pydantic to version 1.x(compatible version of Pydantic)

In [8]:
!pip install pydantic==1.10.7 

Once Pydantic is set to a compatible version, you can also reinstall Gradio to ensure everything works smoothly
If you already have gradio installed, run this code. Otherwise, skip this step.

In [ ]:
!pip uninstall gradio

After installing the correct version of Pydantic and Gradio, restart the Jupyter kernel to apply the changes.

In [11]:
!pip install gradio

  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.7
    Uninstalling pydantic-1.10.7:
      Successfully uninstalled pydantic-1.10.7


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.9.2 which is incompatible.


## Gradio Context:

**Why Gradio?:** 
* When working with machine learning, especially LLMs, you often want an interface to interact with models without writing extensive code. Gradio simplifies this process by allowing developers to quickly build UIs, test models, and share them easily. It's often used in rapid prototyping of models for user feedback or demonstrations.
* You can use Gradio to test LLM outputs in real-time by providing prompts and receiving responses directly on the UI.

In [2]:
# gradio is used for building user-friendly interfaces for ML models. 
# With Gradio, you can quickly create input/output interfaces (for text, images, audio, etc.) for your machine learning models.
import gradio as gr 

**Environment variables** provide a layer of security and flexibility, as sensitive data (API keys, database URLs, etc.) is kept separate from the code. This is essential when dealing with services like OpenAI, Anthropic, or Google, which require API keys for authentication and access.

**LLM Application Workflow:** 

* User sends a query or prompt through a Gradio UI.
* The backend fetches the appropriate API key and sends the request to the LLM provider.
* The LLM processes the prompt and returns a response, which is then displayed in the Gradio interface.

In [3]:
load_dotenv() # load environment variables from a .env file into the program’s environment. 

# os.getenv() looks for an environment variable (like OPENAI_API_KEY). If it exists, the function retrieves its value. 
# If not, it returns the default value provided, ensuring that your program doesn't crash if the .env file is missing.
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [4]:
# Connect to OpenAI, Anthropic and Google

openai = OpenAI() # Initialize the OpenAI client, which will allow you to interact with OpenAI’s API (e.g., for GPT-4 or other models).

# Anthropic() class initializes a connection to Anthropic’s API using the key that was set in the environment variables.
# With this object, you can query models like Claude for tasks such as conversational AI, text generation, and summarization.
claude = anthropic.Anthropic()

google.generativeai.configure()

**System messages** are an essential part of the **prompt engineering** process when interacting with LLMs. They provide context or instructions to shape how the model generates responses. The message usually defines the role or behavior the LLM should adopt while interacting with the user.

**Prompt Engineering:** This technique is crucial when fine-tuning the behavior of LLMs. System messages help ensure that the model adheres to a specific tone, role, or set of expectations. It essentially functions as an invisible guide for the model’s behavior, ensuring that all generated responses stay aligned with the desired assistant-like behavior.

**Role-Playing:** You can change the system message to define different behaviors or roles, such as “You are a sarcastic assistant” or “You are a financial advisor.” This is part of role-playing with LLMs, which is commonly used in chatbot design and conversational AI.

In [5]:
# Define a system message that will guide the behavior of the language model (LLM) in its responses. 
# The LLM interprets this message before processing any user input and uses it as a guiding principle throughout the conversation.
system_message = "You are a helpful assistant"

**System Message:** Guides the behavior of the LLM, making it act like a helpful assistant.

**User Prompt:** The specific input from the user to which the LLM will respond.

**OpenAI API:** The interface used to communicate with and get responses from the GPT models.

In [6]:
def message_gpt(prompt):
    # Create the Message Structure: This message structure is typical in chat-based models like GPT-3.5 or GPT-4, 
    # where multiple roles (system, user, assistant) participate in the conversation.
    messages = [
        {"role": "system", "content": system_message }, # A system message instructs the model how to behave.
        {"role":"user", "content": prompt } # A user message is the actual prompt provided by the user,the specific input to which the model will respond.
    ]
    
    # Calling the GPT-4o-mini Model by using the OpenAI API’s chat.completions.create() method.
    # The openai.chat.completions.create() method is responsible for querying the API and retrieving the completion (the model’s response).
    completion = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages=messages,
    )

    # The API’s response is usually structured into choices. Each choice represents a potential reply generated by the model.
    # The function accesses the first choice (choices[0]) and retrieves the message.content, which contains the generated text.
    return completion.choices[0].message.content


In [7]:
message_gpt("What is today's date?")

"Today's date is October 2, 2023."

Since GPT-4o-mini does not have real-time access to the current date, it will most likely respond with a general answer or an estimate, and it may not give the accurate current date unless it was fine-tuned for that use case.
## To get the actual date:
* If you're looking for real-time functionality like fetching the actual current date, you'll need to integrate Python's datetime module with the function or an external real-time API.

In [8]:

import datetime

def message_gpt(prompt):
    if "date" in prompt.lower():
        current_date = datetime.datetime.now().strftime("%Y-%m-%d")
        return f"Today's date is {current_date}."
    
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

# Example call
response = message_gpt("What is today's date?")
print(response)

Today's date is 2024-10-19.


In [9]:
def shout(text):
    print(f"Shout has been called with input {text}")
    # upper() method is a built-in Python string method that converts all lowercase letters in a string to uppercase.
    return text.upper()  

In [10]:
shout("hello")

Shout has been called with input hello


'HELLO'

 ## **Basic Gradio Interface**
**Purpose:**

This creates a simple Gradio interface that allows users to interact with the shout function through a web-based UI.
The interface has a textbox for input (where users will type in a string) and another textbox for output (which will display the uppercase version of the input).

* **fn=shout:** Specifies that the function to be called when the user submits input is shout.
* **inputs="textbox":** This indicates that the user will provide input through a textbox.
* **outputs="textbox":** The output (uppercased version of the input) will also be displayed in a textbox.
* **.launch():** This launches the Gradio interface, allowing users to interact with it in a web browser. A local web server is started, and a link is provided where the interface can be accessed.
#### Sharing Publicly:
* With **share=True**, Gradio will generate a link (such as https://random-link.gradio.app) that you can share with others, allowing them to interact with your interface without needing to install anything or run the code themselves.

In [11]:
#  Create a simple Gradio interface that allows users to interact with the shout function through a web-based UI.
gr.Interface(fn=shout, 
             inputs = "textbox", 
             outputs = "textbox").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Shout has been called with input What'sup yo
Shout has been called with input i love this tutorial


Disables flagging and provides a public URL for others to test the interface remotely.

In [13]:
# The flagging_mode="never" argument disables the flagging feature. 
# By default, Gradio allows users to flag results if they believe there is an issue (e.g., incorrect output). 
gr.Interface(fn=shout, 
             inputs="textbox", 
             outputs="textbox", 
             flagging_mode="never").launch(share=True)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://3e2f024819c8e7b0ec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Shout has been called with input i love coding, yeah!


In [20]:
view = gr.Interface(
    fn=shout, # Specifie that the shout function will be used
    # The input field is a textbox where the user can type their message. It’s labeled "Your message:" and allows up to 6 lines of text.
    inputs = [gr.Textbox(label = "Your message: ", lines = 6)],
    # The output will be displayed in another textbox, labeled "Response:", with space for 8 lines of text.
    outputs = [gr.Textbox(label = "Response: ", lines = 8)],
    flagging_mode = "never")

view.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Shout has been called with input I love coding, yeah!


## Summary of the Two Interfaces:
* **Interface 1 (shout):** Converts user input to uppercase and displays it in the response box. Useful for simple text transformations.
* **Interface 2 (message_gpt):** Sends user input to the GPT-4o-mini model and displays the AI-generated response. Suitable for interacting with an LLM.

In [21]:
view = gr.Interface(
    fn = message_gpt,
    inputs = [gr.Textbox(label = "Your message: ")],
    outputs = [gr.Textbox(label = "Response: ")],
    flagging_mode = "never"
)

view.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [23]:
system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn = message_gpt,
    inputs = [gr.Textbox(label = "Your message: ")],
    outputs = [gr.Markdown(label = "Response: ")], # The output is now rendered in Markdown format. 
    flagging_mode = "never"
)

view.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


## Yield vs Return:

yield makes this a generator function. Instead of returning a single result after the entire response is generated, it yields intermediate results as they become available.This is what allows the function to stream the results back to the user continuously.

## How It Works in Practice:
When **def stream_gpt(prompt):** is used in a Gradio interface or other interactive environment, the response is displayed to the user incrementally as it’s generated by the GPT-4o-mini model.As soon as the first part of the response is ready, it appears on the screen, followed by subsequent parts, creating a dynamic and responsive experience.

In [33]:
# Streaming GPT-4o-mini Results

def stream_gpt(prompt):
    # Message Structure
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    
    # Calling the GPT-4o-mini API with Streaming
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True # The stream=True argument enables streaming mode
    )
    
    result = "" # result is initialized as an empty string, and each chunk is added to it as the stream progresses.

    # The stream is an iterator that sends chunks of the response back as they are generated.
    for chunk in stream:
        # or "": If delta.content is None (which can happen for some chunks), it adds an empty string instead to avoid errors.
        result += chunk.choices[0].delta.content or ""
        # The yield statement sends back the current state of the result at each step, allowing the function to continuously output the growing result in real time.
        yield result

In [35]:
view = gr.Interface(
    fn = stream_gpt,
    inputs = [gr.Textbox(label = "Your message: ")],
    outputs = [gr.Markdown(label = "Response: ")],
    flagging_mode = "never",
    #live = True # 'live=True' is used for real-time feedback
)

view.launch()

* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


In [30]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model = "claude-3-haiku-20240307",
        max_tokens = 1000,
        temperature = 0.7,
        system = system_message,
        messages = [
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    
    # Set up the streaming of the response, meaning Claude will start generating and sending back parts of the response in real-time.
    with result as stream:
        # As each chunk of the response is streamed back (via stream.text_stream), it is concatenated to the response string. 
        # The text or "" ensures that even if a chunk is empty or None, it won't break the process.
        for text in stream.text_stream:
            response += text or ""
            # The yield response statement sends the current state of the response to the caller at every iteration.
            yield response
        

In [31]:
view = gr.Interface(
    fn = stream_claude,
    inputs = [gr.Textbox(label = "Your message: ")],
    outputs = [gr.Markdown(label = "Response: ")],
    flagging_mode = "never"   
)
view.launch()

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


In [42]:
def stream_model(prompt, model):
    if model == "GPT":
        result = stream_gpt(prompt)
    elif model == "Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
        
    #  Once the appropriate model is selected and called (either GPT or Claude), the response is streamed back in chunks.
    for chunk in result:
        # The yield statement is used to continuously provide the chunks of the result back to the user. 
        # As each chunk of the response is generated, it is yielded, allowing the UI (or any other consumer) to display the response in real time.
        yield chunk

In [43]:
view = gr.Interface(
    fn = stream_model,
    inputs = [gr.Textbox(label = "Your message: "), gr.Dropdown(["GPT","Claude"], label = "Select model")],
    outputs = [gr.Markdown(label = "Response: ")],
    flagging_mode = "never"
)

view.launch()


* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


## Building a company brochure generator

In [45]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text： str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup



SyntaxError: invalid character '：' (U+FF1A) (292981868.py, line 6)